# Machine Learning

## 1. Initial Setup

- Set variables and hyperparameters
- Import libraries
- Load dataset

In [1]:
# Max allowable epochs for each model type
MLP_MAX_EPOCHS = 10
LSTM_MAX_EPOCHS = 10
GRU_MAX_EPOCHS = 10

# Hyperparameters for Hyperparameter Optimization using Optuna
OPTUNA_N_JOBS = 3
OPTUNA_N_TRIALS = 2                # 20
OPTUNA_N_STARTUP_TRIALS = 0         # 5
OPTUNA_WARMUP_STEPS = 0            # 10

In [2]:
PROCESSED_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/processed.csv"

In [3]:
# Python native libraries
from typing import List, Dict, Any, Tuple
import os

# Data manipulation and preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# PyTorch libraries and derivatives
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from torchinfo import summary

# Optuna Libraries
import optuna
from optuna.integration import PyTorchLightningPruningCallback

# Set random seed for reproducibility
L.seed_everything(42, workers=True)

/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Seed set to 42


42

In [4]:
df = pd.read_csv(
    PROCESSED_PATH,
    sep=";",
    parse_dates=["date"],
    dayfirst=True,
)

df.set_index("date", inplace=True)
df.index = pd.to_datetime(df.index)

df.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,flow_after_max,flow_after_min,flow_after_q25,flow_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0.000000,0.0,0.0,0.0,0.0,219.083333,222.0,216.0,217.0,221.00,...,70.8,65.4,66.60,69.60,0.000000,0.008696,0.0,0.000000,0.026087,0.0
2014-01-02,0.008696,0.2,0.0,0.0,0.0,222.956522,226.0,219.0,221.5,224.00,...,85.3,67.8,73.90,78.10,0.008696,0.217391,0.0,0.008696,0.226087,0.0
2014-01-03,0.000000,0.0,0.0,0.0,0.0,225.318182,231.0,219.0,220.5,230.00,...,78.7,63.0,67.95,76.30,0.008696,0.217391,0.0,0.008696,0.217391,0.0
2014-01-04,0.000000,0.0,0.0,0.0,0.0,213.863636,221.0,208.0,212.0,215.75,...,63.0,58.2,61.05,62.85,0.000000,0.000000,0.0,0.008696,0.217391,0.0
2014-01-05,0.000000,0.0,0.0,0.0,0.0,204.904762,208.0,201.0,204.0,206.00,...,58.2,53.4,55.80,57.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0


## 2. Data Preprocessing

- Conferir ausência de dados nulos
- Retirar as colunas de vazão, uma vez que está altamente correlacionada à coluna de nível
- Normalização dos dados

In [5]:
# Count missing values in each column
def print_missing_values(data):
    missing_values = data.isnull().sum()
    print("Missing values in each column:")
    print(missing_values[missing_values > 0])

print_missing_values(df)

Missing values in each column:
Series([], dtype: int64)


In [6]:
# Remove columns containing 'flow' from the dataframe
df = df.loc[:, ~df.columns.str.contains('flow')]
df.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,level_after_max,level_after_min,level_after_q25,level_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0.000000,0.0,0.0,0.0,0.0,219.083333,222.0,216.0,217.0,221.00,...,293.0,284.0,286.00,291.0,0.000000,0.008696,0.0,0.000000,0.026087,0.0
2014-01-02,0.008696,0.2,0.0,0.0,0.0,222.956522,226.0,219.0,221.5,224.00,...,326.0,288.0,298.00,309.0,0.008696,0.217391,0.0,0.008696,0.226087,0.0
2014-01-03,0.000000,0.0,0.0,0.0,0.0,225.318182,231.0,219.0,220.5,230.00,...,306.0,280.0,288.25,302.0,0.008696,0.217391,0.0,0.008696,0.217391,0.0
2014-01-04,0.000000,0.0,0.0,0.0,0.0,213.863636,221.0,208.0,212.0,215.75,...,280.0,271.0,276.75,279.0,0.000000,0.000000,0.0,0.008696,0.217391,0.0
2014-01-05,0.000000,0.0,0.0,0.0,0.0,204.904762,208.0,201.0,204.0,206.00,...,272.0,264.0,268.00,270.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [7]:
scaler = StandardScaler()
df_normalized = pd.DataFrame(
    scaler.fit_transform(df),
    index=df.index,
    columns=df.columns
)
df_normalized.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,level_after_max,level_after_min,level_after_q25,level_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.245701,0.105763,0.354992,0.294064,0.201384,...,0.616501,0.835205,0.781900,0.663710,-0.371786,-0.341985,-0.334039,-0.428193,-0.344398,-0.383211
2014-01-02,-0.20383,-0.200634,0.0,-0.022316,-0.115149,0.301341,0.157947,0.400498,0.360028,0.243211,...,0.911162,0.875776,0.900001,0.829041,-0.318631,0.690675,-0.334039,-0.387379,0.416068,-0.383211
2014-01-03,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.335268,0.223177,0.400498,0.345369,0.326865,...,0.732580,0.794635,0.804044,0.764746,-0.318631,0.690675,-0.334039,-0.387379,0.383004,-0.383211
2014-01-04,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.170716,0.092717,0.233644,0.220771,0.128186,...,0.500423,0.703350,0.690865,0.553489,-0.371786,-0.385012,-0.334039,-0.387379,0.383004,-0.383211
2014-01-05,-0.28545,-0.287747,0.0,-0.022316,-0.115149,0.042016,-0.076881,0.127465,0.103503,-0.007752,...,0.428990,0.632351,0.604750,0.470823,-0.371786,-0.385012,-0.334039,-0.428193,-0.443589,-0.383211


## 3. Criação dos Datasets

- Criar dataset para série temporal
- Dividir o dataset em treino, validação e teste

In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=24):
        self.data = data.reset_index(drop=True)
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data.iloc[
            idx : idx + self.sequence_length
        ].values  # shape: (sequence_length, num_features)
        y = self.data.iloc[idx + self.sequence_length].values  # shape: (num_features,)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(
            y, dtype=torch.float32
        )


class NonOverlappingConcatDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        self.cumulative_lengths = []
        total = 0
        for d in datasets:
            self.cumulative_lengths.append(total)
            total += len(d)
        self.total_length = total

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        # Find which dataset this idx belongs to
        for i in range(len(self.datasets)):
            if idx < self.cumulative_lengths[i] + len(self.datasets[i]):
                local_idx = idx - self.cumulative_lengths[i]
                return self.datasets[i][local_idx]
        raise IndexError("Index out of range")


def split_train_validation_test(
    data: pd.DataFrame, train_size: float = 0.8, val_size: float = 0.1
):
    train_end = int(len(data) * train_size)
    val_end = int(len(data) * (train_size + val_size))

    train_data = data.iloc[:train_end]
    val_data = data.iloc[train_end:val_end]
    test_data = data.iloc[val_end:]

    return train_data, val_data, test_data


def create_datasets(
    sequence_length: int,
) -> Tuple[
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
]:
    datasets_train: List[TimeSeriesDataset] = []
    datasets_validation: List[TimeSeriesDataset] = []
    datasets_test: List[TimeSeriesDataset] = []
    for year in range(2010, 2024):
        year_data = df_normalized[df_normalized.index.year == year].reset_index(
            drop=True
        )
        if not year_data.empty:
            train_data, validation_data, test_data = split_train_validation_test(
                year_data
            )
            datasets_train.append(
                TimeSeriesDataset(train_data, sequence_length=sequence_length)
            )
            datasets_validation.append(
                TimeSeriesDataset(validation_data, sequence_length=sequence_length)
            )
            datasets_test.append(
                TimeSeriesDataset(test_data, sequence_length=sequence_length)
            )

    train_dataset = NonOverlappingConcatDataset(datasets_train)
    validation_dataset = NonOverlappingConcatDataset(datasets_validation)
    test_dataset = NonOverlappingConcatDataset(datasets_test)

    return train_dataset, validation_dataset, test_dataset

In [9]:
# Test for TimeSeriesDataset

# Create a simple DataFrame with increasing integers
test_df = pd.DataFrame({'A': np.arange(10)})

# Window length = 3
ts_dataset = TimeSeriesDataset(test_df, sequence_length=3)

print("Testing TimeSeriesDataset:")
for i in range(len(ts_dataset)):
    x, y = ts_dataset[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")

# Test for NonOverlappingConcatDataset
# Create two small TimeSeriesDatasets
df1 = pd.DataFrame({'A': np.arange(5)})
df2 = pd.DataFrame({'A': np.arange(10, 15)})

ds1 = TimeSeriesDataset(df1, sequence_length=2)
ds2 = TimeSeriesDataset(df2, sequence_length=2)

concat_ds = NonOverlappingConcatDataset([ds1, ds2])

print("\nTesting NonOverlappingConcatDataset")
print("Should not overlap and should concatenate correctly:")
for i in range(len(concat_ds)):
    x, y = concat_ds[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")


Testing TimeSeriesDataset:
Index 0: x = [0. 1. 2.], y = [3.]
Index 1: x = [1. 2. 3.], y = [4.]
Index 2: x = [2. 3. 4.], y = [5.]
Index 3: x = [3. 4. 5.], y = [6.]
Index 4: x = [4. 5. 6.], y = [7.]
Index 5: x = [5. 6. 7.], y = [8.]
Index 6: x = [6. 7. 8.], y = [9.]

Testing NonOverlappingConcatDataset
Should not overlap and should concatenate correctly:
Index 0: x = [0. 1.], y = [2.]
Index 1: x = [1. 2.], y = [3.]
Index 2: x = [2. 3.], y = [4.]
Index 3: x = [10. 11.], y = [12.]
Index 4: x = [11. 12.], y = [13.]
Index 5: x = [12. 13.], y = [14.]


## 4. Definição dos Modelos

In [10]:
class MLP(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_layers,
        sequence_length=5,
        learning_rate: float = 0.0001,
    ):
        super(MLP, self).__init__()
        self.save_hyperparameters()
        self.sequence_length = sequence_length

        # MLP para cada passo temporal
        layers = []
        in_features = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())
            in_features = hidden_size
        self.step_mlp = nn.Sequential(*layers)
        self.final_layer = nn.Linear(in_features, input_size)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        # x: (batch, window_length, input_size)
        # Aplica o MLP em cada passo temporal    print("x type:", type(x))
        batch, window, features = x.shape
        x = x.view(-1, features)  # (batch * window_length, input_size)
        out = self.step_mlp(x)  # (batch * window_length, hidden)
        out = out.view(batch, window, -1)  # (batch, window_length, hidden)
        out = out.mean(dim=1)  # (batch, hidden) - agregação temporal
        out = self.final_layer(out)  # (batch, input_size)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class LSTM(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(LSTM, self).__init__()
        self.save_hyperparameters()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class GRU(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(GRU, self).__init__()
        self.save_hyperparameters()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


print(summary(MLP(input_size=5, hidden_layers=[10, 10])))
print("\n")
print(summary(LSTM(input_size=5, hidden_size=10, num_layers=2)))
print("\n")
print(summary(GRU(input_size=5, hidden_size=10, num_layers=2)))

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       60
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       110
│    └─ReLU: 2-4                         --
├─Linear: 1-2                            55
├─MSELoss: 1-3                           --
Total params: 225
Trainable params: 225
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              1,560
├─Linear: 1-2                            55
├─MSELoss: 1-3                           --
Total params: 1,615
Trainable params: 1,615
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
GRU                                      --
├─GRU: 1-1                               1,170
├─Linear: 1-2                            55
├─MSELoss: 1-3                           --
Total 

## 5. Treinamento dos Modelos

### 5.1. training functions

In [14]:
def train_model(
    model,
    train_data,
    val_data,
    logger,
    epochs,
    batch_size,
    callbacks=[],
    verbose=True,
) -> Tuple[float, str]:
    """
    Treina o modelo com os dados de treino e validação.
    """

    train_loader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True, drop_last=True
    )
    validation_loader = DataLoader(
        val_data, batch_size=4, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=verbose,
        log_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        check_val_every_n_epoch=1,
        enable_model_summary=verbose,
    )
    trainer.fit(model, train_loader, validation_loader)


def evaluate_model(model, test_data, logger, batch_size=4):
    """
    Avalia o modelo com os dados de teste.
    """
    test_loader = DataLoader(
        test_data, batch_size=batch_size, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=True,
        deterministic=True,
    )
    results = trainer.test(model, test_loader)

    return results


def run_experiment(
    model_class: type[L.LightningModule],
    name: str,
    model_kwargs: Dict[str, Any] = {},
    train_kwargs: Dict[str, Any] = {},
    evaluate: bool = True,
    callbacks: List[L.Callback] = [],
    verbose: bool = True,
):
    """
    Executa um experimento de treinamento e avaliação do modelo.
    """
    # Logger
    logger = TensorBoardLogger(save_dir=os.getcwd(), name=f"lightning_logs/{name}")

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss", mode="min", save_top_k=1, filename="best_model"
    )

    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,
        patience=10,
        mode="min",
        strict=False,
    )

    train_dataset, validation_dataset, test_dataset = create_datasets(
        sequence_length=train_kwargs.get("sequence_length", 5)
    )
    
    # Remove 'sequence_length' from train_kwargs if present
    train_kwargs = dict(train_kwargs)  # make a copy to avoid side effects
    train_kwargs.pop("sequence_length", None)
    
    input_size = train_dataset[0][0].shape[1]  # Assuming all inputs have the same shape

    model = model_class(input_size=input_size, **(model_kwargs or {}))
    
    train_model(
        model,
        train_dataset,
        validation_dataset,
        logger=logger,
        **(train_kwargs or {}),
        callbacks=[checkpoint_callback, early_stopping_callback, *callbacks],
        verbose=verbose,
    )

    best_loss = checkpoint_callback.best_model_score
    best_model_path = checkpoint_callback.best_model_path

    # To view the logs, you can use TensorBoard.
    # If 'logs' is a SummaryWriter, get its log_dir and launch TensorBoard:
    print(f"To view logs, run in terminal:\n  tensorboard --logdir {logger.log_dir}")

    print("Best model saved at:", best_model_path)

    eval_results = {}
    if evaluate:
        best_model = model_class.load_from_checkpoint(best_model_path, **model_kwargs)
        eval_results = evaluate_model(best_model, test_dataset, logger, batch_size=4)

        print("Evaluation results:", eval_results)

    return {
        "results": eval_results,
        "best_loss": best_loss,
        "best_model_path": best_model_path,
    }

### 5.2. Hyperparameter optimization functions

In [19]:
import warnings


def get_optimization_args(model_class: L.LightningModule, trial):
    """
    Retorna os argumentos necessários para executar o experimento.
    """
    if model_class == MLP:
        model_kwargs = {
            "hidden_layers": trial.suggest_categorical(
                "hidden_layers", ([32, 16], [48, 24], [64, 32], [96, 48], [128, 64])
            ),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),
        }
        train_kwargs = {
            "epochs": MLP_MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 8, 32),
            "sequence_length": trial.suggest_int("sequence_length", 3, 12),
        }
    elif model_class == LSTM:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-2),
        }
        train_kwargs = {
            "epochs": LSTM_MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 8, 32),
            "sequence_length": trial.suggest_int("sequence_length", 3, 12),
        }
    elif model_class == GRU:
        model_kwargs = {
            "hidden_size": trial.suggest_categorical("hidden_size", [16, 32, 64]),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),
        }
        train_kwargs = {
            "epochs": GRU_MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 8, 32),
            "sequence_length": trial.suggest_int("sequence_length", 3, 12),
        }
    else:
        raise ValueError(f"Unsupported model class: {model_class}")
    return model_kwargs, train_kwargs


def get_objective_fn(model_class, name):
    """
    Retorna uma função de objetivo para otimização de hiperparâmetros.
    """

    def objective(trial: optuna.trial.Trial) -> float:
        # Ajusta os hiperparâmetros com base no trial
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            model_kwargs, train_kwargs = get_optimization_args(model_class, trial)

        optuna_pruning_callback = PyTorchLightningPruningCallback(
            trial, monitor="val_loss"
        )

        results = run_experiment(
            model_class=model_class,
            name=name,
            model_kwargs=model_kwargs,
            train_kwargs=train_kwargs,
            evaluate=False,
            callbacks=[optuna_pruning_callback],
            verbose=False,
        )
        return results["best_loss"]

    return objective


def run_optimization(objective_fn, n_trials=OPTUNA_N_TRIALS) -> optuna.study.Study:
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=OPTUNA_N_STARTUP_TRIALS, n_warmup_steps=OPTUNA_WARMUP_STEPS
    )
    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective_fn, n_trials=n_trials)

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    return study

### 5.3. Training loop

In [20]:
mlp_objective_fn = get_objective_fn(
    MLP,
    "mlp",
)

study_mlp = run_optimization(mlp_objective_fn, n_trials=OPTUNA_N_TRIALS)

mlp_optimized_results = run_experiment(
    MLP,
    "mlp_optimized",
    model_kwargs={
        "hidden_layers": study_mlp.best_trial.params["hidden_layers"],
        "learning_rate": study_mlp.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MLP_MAX_EPOCHS,
        "batch_size": study_mlp.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)

print("\nMLP Optimized Results:", mlp_optimized_results)

[I 2025-06-03 18:54:24,172] A new study created in memory with name: no-name-2d4a3cd2-6a60-4548-a25c-c7989816ab7e
/tmp/ipykernel_174663/1535762886.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/c

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp/version_72
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp/version_72/checkpoints/best_model.ckpt


[I 2025-06-03 18:54:34,406] Trial 1 pruned. Trial was pruned at epoch 0.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | step_mlp    | Sequential | 8.2 K  | train
1 | final_layer | Linear     | 1.8 K  | train
2 | criterion   | MSELoss    | 0      | train
---------------------------------------------------
10.0 K    Trainable params
0         Non-trainable params
10.0 K    Total params
0.040     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Best trial:
  Value: 0.8722650408744812
  Params: 
    hidden_layers: [96, 48]
    learning_rate: 0.00415880289441692
    batch_size: 10
    sequence_length: 11


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp_optimized/version_5
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp_optimized/version_5/checkpoints/best_model.ckpt


/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.8500663042068481     │
└───────────────────────────┴───────────────────────────┘

Evaluation results: [{'test_loss': 0.8500663042068481}]

MLP Optimized Results: {'results': [{'test_loss': 0.8500663042068481}], 'best_loss': tensor(0.7544, device='cuda:0'), 'best_model_path': '/home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/mlp_optimized/version_5/checkpoints/best_model.ckpt'}


In [21]:
lstm_objective_fn = get_objective_fn(
    LSTM,
    "lstm",
)
study_lstm = run_optimization(lstm_objective_fn, n_trials=OPTUNA_N_TRIALS)
lstm_optimized_results = run_experiment(
    LSTM,
    "lstm_optimized",
    model_kwargs={
        "hidden_size": study_lstm.best_trial.params["hidden_size"],
        "num_layers": study_lstm.best_trial.params["num_layers"],
        "learning_rate": study_lstm.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": LSTM_MAX_EPOCHS,
        "batch_size": study_lstm.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nLSTM Optimized Results:", lstm_optimized_results)

[I 2025-06-03 18:54:52,665] A new study created in memory with name: no-name-f53f03ad-0947-4ae8-bda5-cd24e0f8f325
/tmp/ipykernel_174663/1535762886.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-2),
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/c

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm/version_5
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm/version_5/checkpoints/best_model.ckpt


`Trainer.fit` stopped: `max_epochs=10` reached.
[I 2025-06-03 18:55:06,489] Trial 1 finished with value: 0.717122495174408 and parameters: {'hidden_size': 74, 'num_layers': 1, 'learning_rate': 0.0005183682583086482, 'batch_size': 17, 'sequence_length': 7}. Best is trial 1 with value: 0.717122495174408.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | lstm      | LSTM    | 33.2 K | train
1 | fc        | Linear  | 2.7 K  | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
35.9 K    Trainable params
0         Non-trainable params
35.9 K    Total params
0.143     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm/version_6
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm/version_6/checkpoints/best_model.ckpt
Best trial:
  Value: 0.717122495174408
  Params: 
    hidden_size: 74
    num_layers: 1
    learning_rate: 0.0005183682583086482
    batch_size: 17
    sequence_length: 7


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm_optimized/version_2
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm_optimized/version_2/checkpoints/best_model.ckpt


/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.7015186548233032     │
└───────────────────────────┴───────────────────────────┘

Evaluation results: [{'test_loss': 0.7015186548233032}]

LSTM Optimized Results: {'results': [{'test_loss': 0.7015186548233032}], 'best_loss': tensor(0.6947, device='cuda:0'), 'best_model_path': '/home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/lstm_optimized/version_2/checkpoints/best_model.ckpt'}


In [22]:
gru_objective_fn = get_objective_fn(
    GRU,
    "gru",
)
study_gru = run_optimization(gru_objective_fn, n_trials=OPTUNA_N_TRIALS)
gru_optimized_results = run_experiment(
    GRU,
    "gru_optimized",
    model_kwargs={
        "hidden_size": study_gru.best_trial.params["hidden_size"],
        "num_layers": study_gru.best_trial.params["num_layers"],
        "learning_rate": study_gru.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": GRU_MAX_EPOCHS,
        "batch_size": study_gru.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nGRU Optimized Results:", gru_optimized_results)

[I 2025-06-03 18:55:20,237] A new study created in memory with name: no-name-36fbbd45-fb38-4c08-ab89-32210f991f18
/tmp/ipykernel_174663/1535762886.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/c

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru/version_3
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru/version_3/checkpoints/best_model.ckpt


[I 2025-06-03 18:55:27,905] Trial 1 pruned. Trial was pruned at epoch 0.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | gru       | GRU     | 44.5 K | train
1 | fc        | Linear  | 2.3 K  | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
46.9 K    Trainable params
0         Non-trainable params
46.9 K    Total params
0.188     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Best trial:
  Value: 0.7117761969566345
  Params: 
    hidden_size: 64
    num_layers: 2
    learning_rate: 0.0012781362861554034
    batch_size: 25
    sequence_length: 6


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

To view logs, run in terminal:
  tensorboard --logdir /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru_optimized/version_1
Best model saved at: /home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru_optimized/version_1/checkpoints/best_model.ckpt


/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.7163823843002319     │
└───────────────────────────┴───────────────────────────┘

Evaluation results: [{'test_loss': 0.7163823843002319}]

GRU Optimized Results: {'results': [{'test_loss': 0.7163823843002319}], 'best_loss': tensor(0.6924, device='cuda:0'), 'best_model_path': '/home/bruno/Workspace/FIAP/2025/gs-disaster/src/lightning_logs/gru_optimized/version_1/checkpoints/best_model.ckpt'}
